In [1]:
# This line configures matplotlib to show figures embedded in the notebook.
%matplotlib inline
from sklearn.datasets import load_boston
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import matplotlib.cm as cm
from matplotlib.colors import Normalize, rgb2hex
import seaborn as sns
import folium

Source: [CDC: Chronic Disease Indicators](https://www.cdc.gov/cdi/index.html)

In [2]:
usecols = ['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'DataValueUnit', 'DataValueType',
       'DataValue', 'DataValueAlt', 'DataValueFootnoteSymbol',
       'DatavalueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit',
       'StratificationCategory1', 'Stratification1', 
       'GeoLocation', 'LocationID', 'TopicID', 'QuestionID',
       'DataValueTypeID', 'StratificationCategoryID1', 'StratificationID1' ]

df = pd.read_csv("U.S._Chronic_Disease_Indicators__CDI_.csv", usecols=usecols, dtype={
    'DataValue'                 : 'object',
    'StratificationCategoryID1' : 'category',
    'StratificationID1'         : 'category',
    'DataValueTypeID'           : 'category',
    'QuestionID'                : 'category',
    'TopicID'                   : 'category',
})

df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,...,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
0,2016,2016,US,United States,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,%,Crude Prevalence,16.9,...,18.0,Overall,Overall,NaN,59,ALC,ALC2_2,CrdPrev,OVERALL,OVR
1,2016,2016,AL,Alabama,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,%,Crude Prevalence,13.0,...,14.1,Overall,Overall,"(32.84057112200048, -86.63186076199969)",1,ALC,ALC2_2,CrdPrev,OVERALL,OVR
2,2016,2016,AK,Alaska,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,%,Crude Prevalence,18.2,...,20.6,Overall,Overall,"(64.84507995700051, -147.72205903599973)",2,ALC,ALC2_2,CrdPrev,OVERALL,OVR
3,2016,2016,AZ,Arizona,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,%,Crude Prevalence,15.6,...,16.9,Overall,Overall,"(34.865970280000454, -111.76381127699972)",4,ALC,ALC2_2,CrdPrev,OVERALL,OVR
4,2016,2016,AR,Arkansas,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,%,Crude Prevalence,15.0,...,17.2,Overall,Overall,"(34.74865012400045, -92.27449074299966)",5,ALC,ALC2_2,CrdPrev,OVERALL,OVR


In [3]:
# Extract location-related attributes into a location table.
dfLocs = df[['LocationID', 'GeoLocation', 'LocationAbbr', 'LocationDesc']]\
    .set_index('LocationID').groupby('LocationID').first()
# Split GeoLocation string into Latitude and Longitude floats
geoLocs = dfLocs['GeoLocation'].str[1:-1].str.split(',', expand=True).astype('float')
geoLocs.columns=['Latitude', 'Longitude']
# Combine Lat/Long into Locations table, and drop unused column.
dfLocs = (pd.concat([dfLocs, geoLocs], axis=1)).drop('GeoLocation', axis=1)
dfLocs.head()

,LocationAbbr,LocationDesc,Latitude,Longitude
LocationID,,,,
1,AL,Alabama,32.840571,-86.631861
2,AK,Alaska,64.845080,-147.722059
4,AZ,Arizona,34.865970,-111.763811
5,AR,Arkansas,34.748650,-92.274491
6,CA,California,37.638640,-121.000000


In [4]:
# Extract topic attributes into a topic table.
dfTopics = df[['TopicID', 'Topic']].groupby('TopicID').first()
dfTopics.head()

,Topic
TopicID,
ALC,Alcohol
ART,Arthritis
AST,Asthma
CAN,Cancer
DIA,Diabetes


In [5]:
# Extract question attributes into a question table.
dfQuestions = df[['QuestionID', 'Question']].groupby('QuestionID').first()
dfQuestions.head()

,Question
QuestionID,
ALC2_2,Binge drinking prevalence among adults aged >=...
ALC2_3,Binge drinking prevalence among women aged 18-...
ALC3_0,Binge drinking frequency among adults aged >= ...
ALC4_0,Binge drinking intensity among adults aged >= ...
ALC5_1,Heavy drinking among adults aged >= 18 years


In [6]:
# Extract Stratification attributes into a Stratification table.
dfStrats = df[['StratificationID1', 'Stratification1']].groupby('StratificationID1').first()
dfStrats

,Stratification1
StratificationID1,
BLK,"Black, non-Hispanic"
GENF,Female
GENM,Male
HIS,Hispanic
MRC,"Multiracial, non-Hispanic"
OTH,"Other, non-Hispanic"
OVR,Overall
WHT,"White, non-Hispanic"
AIAN,American Indian or Alaska Native


In [7]:
# Extract StratificationCategory attributes into a StratificationCategory table.
dfStratCats = df[['StratificationCategoryID1', 'StratificationCategory1']] \
    .groupby('StratificationCategoryID1').first()
dfStratCats

,StratificationCategory1
StratificationCategoryID1,
GENDER,Gender
OVERALL,Overall
RACE,Race/Ethnicity


In [8]:
# Extract DataValueType attributes into a DataValueType table.
dfDataValueTypes = df[['DataValueTypeID', 'DataValueType', 'DataValueUnit']]\
    .groupby('DataValueTypeID').first()
dfDataValueTypes

,DataValueType,DataValueUnit
DataValueTypeID,,
AgeAdjMean,Age-adjusted Mean,Number
AgeAdjPrev,Age-adjusted Prevalence,%
CrdPrev,Crude Prevalence,%
Mean,Mean,Number
Nmbr,Number,"cases per 1,000,000"
Median,Median,Number
AgeAdjRate,Age-adjusted Rate,"cases per 10,000"
AgeSexRaceAdjRate,"Adjusted by age, sex, race and ethnicity","cases per 1,000,000"
CrdRate,Crude Rate,"cases per 10,000"


In [9]:
dfData = df.drop(['GeoLocation', 'LocationAbbr', 'LocationDesc', 'Topic', 'Question', 
    'Stratification1', 'StratificationCategory1', 'DataValueType', 'DataValueUnit'], axis=1)
dfData.head()
dfData[dfData['DataValue'] == 'No']

,YearStart,YearEnd,DataSource,DataValue,DataValueAlt,DataValueFootnoteSymbol,DatavalueFootnote,LowConfidenceLimit,HighConfidenceLimit,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
242012,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,9,TOB,TOB7_0,YesNo,OVERALL,OVR
242053,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,11,TOB,TOB7_0,YesNo,OVERALL,OVR
242074,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,10,TOB,TOB7_0,YesNo,OVERALL,OVR
242643,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,12,TOB,TOB7_0,YesNo,OVERALL,OVR
242689,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,13,TOB,TOB7_0,YesNo,OVERALL,OVR
242933,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,15,TOB,TOB7_0,YesNo,OVERALL,OVR
242956,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,19,TOB,TOB7_0,YesNo,OVERALL,OVR
243137,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,16,TOB,TOB7_0,YesNo,OVERALL,OVR
243160,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,17,TOB,TOB7_0,YesNo,OVERALL,OVR
243360,2015,2015,STATE,No,NaN,NaN,NaN,NaN,NaN,18,TOB,TOB7_0,YesNo,OVERALL,OVR


<img src="indicator_11_3.png" alt="Prevalence of depressive disorders among adults
aged ≥18 years with diagnosed diabetes" title="Indicator 11.3" />

In [10]:
# Drill into indicator 11.3
df11_3 = dfData[df['QuestionID'] == 'DIA11_3']
# Strip out NaN data values
df11_3 = df11_3[(df11_3['DataValueAlt'].notnull()) &
                (df11_3['StratificationCategoryID1'] == 'OVERALL') &
                #(df11_3['LocationID'] == 1) &
                #(df11_3['YearStart'] == 2016) &
                (df11_3['DataValueTypeID'] == 'AgeAdjPrev')]
df11_3 = df11_3.groupby('LocationID').mean()
df11_3 = pd.merge(df11_3, dfLocs, left_index=True, right_index=True)
# Strip out U.S. general location (no geo location info)
df11_3 = df11_3[df11_3['Latitude'].notnull()]
df11_3.head()

,YearStart,YearEnd,DataValueAlt,LowConfidenceLimit,HighConfidenceLimit,LocationAbbr,LocationDesc,Latitude,Longitude
LocationID,,,,,,,,,
1,2013.5,2013.5,36.450000,29.766667,43.733333,AL,Alabama,32.840571,-86.631861
2,2013.5,2013.5,30.500000,20.000000,43.300000,AK,Alaska,64.845080,-147.722059
4,2013.5,2013.5,35.833333,27.966667,44.566667,AZ,Arizona,34.865970,-111.763811
5,2013.5,2013.5,38.216667,29.500000,47.766667,AR,Arkansas,34.748650,-92.274491
6,2013.5,2013.5,19.583333,14.716667,25.566667,CA,California,37.638640,-121.000000


In [11]:
norm = Normalize(df11_3['DataValueAlt'].min(), df11_3['DataValueAlt'].max())

fmap = folium.Map(location=[48, -102], zoom_start=3)
for loc in df11_3.iterrows():
    val = loc[1]['DataValueAlt']
    color = rgb2hex(cm.OrRd(norm(val)))
    
    folium.CircleMarker([loc[1]['Latitude'], loc[1]['Longitude']], 
                    popup="%s: %0.02f%%" % (loc[1]['LocationDesc'], val), 
                    radius=0.25*val,
                    color=color,
                    fill=color).add_to(fmap)
fmap